In [2]:
from langgraph.graph import END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnablePassthrough
from typing import TypedDict
from app.utils.converters import to_path_map

class State(TypedDict):
    current_step: int
    total_number_of_steps: int

g = StateGraph(State)
g.set_entry_point("entry")

g.add_node("entry", RunnablePassthrough())
g.add_edge("entry", "A")

g.add_node("A", RunnablePassthrough())
g.add_edge("A", "B")

g.add_node("B", RunnablePassthrough())
g.add_edge("B", "reduce_middle_step")

g.add_node("reduce_middle_step", RunnablePassthrough())
g.add_conditional_edges(
    "reduce_middle_step",
    lambda state: (
        "__end__" if state["current_step"] >= state["total_number_of_steps"] else "A"
    ),
    to_path_map([
        END, "A"
    ])
)

main_graph = g.compile(checkpointer=MemorySaver(), interrupt_after=["B"])


with open("./main_graph.png", "wb") as f:
    f.write(main_graph.get_graph().draw_mermaid_png())

In [3]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_dir="/Users/minkijung/Documents/2PetProjects/AI_README_Generator/backend/cache/cloned_repositories/GitMeetup",
    required_exts=[".py"],
    recursive=True,
).load_data()

In [4]:
print(len(documents))

1
